Import libraries

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import joblib
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input, Bidirectional, Attention, GlobalAveragePooling1D, Conv1D, MaxPooling1D
from tensorflow.keras.optimizers import Adam
import seaborn as sns

Define output paths

In [ ]:
ModelPath = "YOUR_PATH"
jobLibPath = "YOUR_PATH"

Set training configuration

In [ ]:
stock_symbol = "^IXIC"
start_date, end_date = "2010-01-01", "2020-07-01"
window_size = 60

Download and preprocess data

In [ ]:
data = yf.download(stock_symbol, start=start_date, end=end_date)
prices = data[['Close']].values
train_size = int(len(prices) * 0.8)

scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(prices[:train_size])
scaled_data = scaler.transform(prices)

def create_dataset(dataset, time_step=60):
    x, y = [], []
    for i in range(time_step, len(dataset)):
        x.append(dataset[i-time_step:i, 0])
        y.append(dataset[i, 0])
    return np.array(x), np.array(y)

x_train, y_train = create_dataset(scaled_data[:train_size], window_size)
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)

Build and compile model

In [ ]:
inputs = Input(shape=(window_size, 1))

x = Conv1D(filters=64, kernel_size=3, activation='relu')(inputs)
x = MaxPooling1D(pool_size=2)(x)

x = Bidirectional(LSTM(100, return_sequences=True))(x)
x = Dropout(0.2)(x)

query = Dense(128)(x)
value = Dense(128)(x)
att_out = Attention()([query, value])

avg_pool = GlobalAveragePooling1D()(att_out)
z = Dense(32, activation='relu')(avg_pool)
output = Dense(1)(z)

model = Model(inputs=inputs, outputs=output)
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

Train model

In [ ]:
print(f"Training momentum-optimized model for NASDAQ...")
model.fit(x_train, y_train, epochs=60, batch_size=32, verbose=0)

Save model and scaler

In [ ]:
model.save(ModelPath)
joblib.dump(scaler, jobLibPath)
print("Saved: model_nasdaq_hybrid.keras")